In [24]:
# In[] model load
import pandas as pd
import panel as pn
# import dask.dataframe as dd
import numpy as np
# from io import BytesIO
import random
import plotly.express as px
import plotly.io as pio
import random

css = ['https://cdn.datatables.net/1.10.24/css/jquery.dataTables.min.css',
       # Below: Needed for export buttons
       'https://cdn.datatables.net/buttons/1.7.0/css/buttons.dataTables.min.css'
]
js = {
    '$': 'https://code.jquery.com/jquery-3.5.1.js',
    'DataTable': 'https://cdn.datatables.net/1.10.24/js/jquery.dataTables.min.js',
    # Below: Needed for export buttons
    'buttons': 'https://cdn.datatables.net/buttons/1.7.0/js/dataTables.buttons.min.js',
    'jszip': 'https://cdnjs.cloudflare.com/ajax/libs/jszip/3.1.3/jszip.min.js',
    'pdfmake': 'https://cdnjs.cloudflare.com/ajax/libs/pdfmake/0.1.53/pdfmake.min.js',
    'vfsfonts': 'https://cdnjs.cloudflare.com/ajax/libs/pdfmake/0.1.53/vfs_fonts.js',
    'html5buttons': 'https://cdn.datatables.net/buttons/1.7.0/js/buttons.html5.min.js',
}
pio.renderers.default='notebook'
pn.extension('plotly',sizing_mode='stretch_width',css_files=css, js_files=js)


## 기본 widget

### One site

In [25]:
chemical = '화학물질1'
# --> 모델 돌려서 시나리오별 농도값 산출
# In[]
# 1. receptor df 만들기
receptor_df = pd.DataFrame(columns=['receptor','group ID', 'individual ID', 'age', 'home', 'other', 'home time', 'other time'])

receptor_df['receptor'] = np.arange(1, 101, 1)

a = np.array([])
s = 1
for i in range(1,101):
    a = np.append(a, s)
    if i % 3 == 0:
        s = s + 1

receptor_df['group ID'] = a
receptor_df['individual ID'] = np.arange(1, 101, 1)
receptor_df['age'] = np.random.choice([1,2,3,4,5], 100)

h_scenario_list = np.array(['h1', 'h2', 'h3', 'h4'])
o_scenario_list = np.array(['o1', 'o2', 'o3', 'o4', 'o5', 'o6', 'o7', 'o8', 'o9', 'o10'])
scenario_list = np.concatenate((h_scenario_list, o_scenario_list))

receptor_df['home'] = np.random.choice(h_scenario_list, 100)
receptor_df['other'] = np.random.choice(o_scenario_list, 100)
receptor_df['home time'] = np.random.uniform(low=0.3, high=0.5, size = 100)
receptor_df['other time'] = np.random.uniform(low=0, high=0.5, size = 100) * (5/7)

#%% example
    
source_index = np.arange(0,20)
source_list =[]
for i in source_index:
    name = '배출원'+str(i)
    source_list.append(name)

chemical_index =np.arange(0,8)
chemical_list = []
for i in chemical_index:
    name = '화학물질'+str(i)
    chemical_list.append(name)

dt = np.random.rand(len(chemical_list),3)
materials= pd.DataFrame(data = dt, columns=['hm','kp','kdust'], index = chemical_list) # 물질특성 hm ,kp, kdust

dt2 =np.random.rand(len(scenario_list),len(source_list))
spaces = pd.DataFrame(data =dt2, index = scenario_list,columns = source_list) # A_i

bot_A =np.random.rand(len(scenario_list))
botA = pd.DataFrame(data=bot_A, index = scenario_list, columns = ['bottomA']) # A_bottom
 
km_dt = np.random.rand(len(scenario_list),len(chemical_list))
kma = pd.DataFrame(data= km_dt, columns = chemical_list, index = scenario_list) # kma

Q_dt =np.random.rand(len(scenario_list))
Q = pd.DataFrame(data=Q_dt, columns=['Q'], index = scenario_list)
#%%  분자 hm*sigma_i(A) * kma * C0
C0 =2

sigmaA = spaces.sum(axis=1)
hm = materials['hm']
rows1 = np.array(sigmaA).reshape(len(sigmaA),1)
cols1 =  np.array(hm).reshape(len(hm),1)

 # sigmaA * hm  , elementarywise product
n1 =np.matmul( rows1, cols1.T)

 # sigmaA * hm* kma , elementarywise product
numerator = C0*np.multiply(n1, kma )

#%% 분모
tsp = 1.5
vt = 0.6
kp = materials['kp']

# tsp*Q*kp , elementarywise product , 14x8
term1 = tsp*np.multiply( np.array(Q), np.array(kp).T) 
 # 14x8
term2 = vt*tsp*np.multiply(np.array(botA), np.array(kp).T)

Q_expand = np.full( len(chemical_list),1)

term0 =np.multiply( np.array(Q), Q_expand.T) #14x8


denomitor = term0 + term1 + term2
#%% 값
conc_df = numerator/denomitor
conc_df = conc_df[chemical]

# In[] 2. AR matrix 만들기

AR_df = pd.DataFrame(columns=scenario_list, index = range(100))
AR_df = AR_df.fillna(0)

for i in range(100):
    for j in range(len(AR_df.columns)):
        if receptor_df['home'][i] == AR_df.columns[j]:
            AR_df.iloc[i,j] = receptor_df['home time'][i]
        if receptor_df['other'][i] == AR_df.columns[j]:
            AR_df.iloc[i,j] = receptor_df['other time'][i]

# 추가로 가구단위로 쓰는것은 가구원 수로 나누기
# In[] 3. 연산
result_df = pd.DataFrame(columns=['individual ID', 'age', 'exposure'])
result_df['individual ID'] = receptor_df['individual ID'] 
result_df['age'] = receptor_df['age']
result_df['exposure'] = np.matmul(AR_df, conc_df.T)

#%% 기여율
weight = spaces.div(sigmaA, axis = 0)

# 연령별 평균값 table
result1 = result_df.groupby(['age'], as_index=False)['exposure'].mean()
# 연령별 중앙값 table
result2 = result_df.groupby(['age'], as_index=False)['exposure'].median()
# 연령별 5 percentile
result3 = result_df.groupby(['age'], as_index=False)['exposure'].quantile(0.05)
# 연령별 95 percentile
result4 = result_df.groupby(['age'], as_index=False)['exposure'].quantile(0.95)

In [26]:
table = pd.DataFrame(columns=['연령', '노출량'])
table['연령'] = ['영아','유아','미취학아동','취학아동','성인','전체']

table['노출량'] = result1['exposure'].round(2).astype(str) + ' (± ' + result2['exposure'].round(2).astype(str) + ')'

table['노출량'][5] = str(round(result_df['exposure'].mean(),2)) + ' (± ' + str(round(result_df['exposure'].std(),2)) + ')'

table=table.style.hide_index()
table_1=pn.Column(table,width=750,height=200)

In [27]:
# 어린이집
table_n = pd.DataFrame(columns=['시나리오', '예측된 공기 중 농도 (㎍/㎥)', '예측된 먼지 농도 (㎍/g)'])
table_n['시나리오'] = ['바닥재(PVC), 실크벽지, 의자(mdf), 선반(mdf)', '바닥재(나무), 종이벽지, 의자(mdf), 선반(mdf)', '바닥재(PVC), 종이벽지, 의자(mdf), 선반(mdf)', '바닥재(나무), 실크벽지, 의자(mdf), 선반(mdf)']
table_n['예측된 공기 중 농도 (㎍/㎥)'] = np.array(conc_df.loc['o1':'o4']).flatten()
table_n['예측된 먼지 농도 (㎍/g)'] = np.array(conc_df.loc['o1':'o4']).flatten()
table_2=pn.Column(table_n,width=1000,height=200)

# 가정
table_h = pd.DataFrame(columns=['시나리오', '예측된 공기 중 농도 (㎍/㎥)', '예측된 먼지 농도 (㎍/g)'])
table_h['시나리오'] = ['바닥재(pvc, DINCH), 실크벽지, 신발장 1개, 책상 1개, 의자1개, 서랍 1개, 침대1,  부엌가구, 식탁, 의자2, 전자제품(나연제,voc)', 
                '바닥재(pvc, DINCH), 실크벽지, 신발장 1개, 책상 1개, 의자1개, 서랍 2개, 침대2,  부엌가구2, 식탁, 의자4, 전자제품(나연제,voc)', 
                '바닥재(pvc, DEHP), 실크벽지, 신발장 1개, 책상 1개, 의자1개, 서랍 1개, 침대1,  부엌가구, 식탁, 의자2, 전자제품(나연제,voc)', 
                '바닥재(pvc, DEHP), 실크벽지, 신발장 1개, 책상 1개, 의자1개, 서랍 2개, 침대2,  부엌가구2, 식탁, 의자4, 전자제품(나연제,voc)']
table_h['예측된 공기 중 농도 (㎍/㎥)'] = np.array(conc_df.loc['h1':'h4']).flatten()
table_h['예측된 먼지 농도 (㎍/g)'] = np.array(conc_df.loc['h1':'h4']).flatten()
table_3=pn.Column(table_h,width=1000,height=200)

# 학교교실
table_s = pd.DataFrame(columns=['시나리오', '예측된 공기 중 농도 (㎍/㎥)', '예측된 먼지 농도 (㎍/g)'])
table_s['시나리오'] = ['바닥재:대리석, 벽:페인트, 학생용책상(mdf) 24개, 의자 (mdf) 24개, 사물함 (철재) 24개', 
                '바닥재:대리석, 벽:페인트, 학생용책상(mdf) 24개, 의자 (mdf) 24개, 사물함(mdf)24개',
                '바닥재:PVC, 벽:페인트, 학생용책상(mdf) 24개, 의자 (mdf) 24개, 사물함 (mdf) 24개',
                '바닥재:PVC, 벽:페인트, 학생용책상(mdf) 24개, 의자 (mdf) 24개, 사물함(철재)24개']
table_s['예측된 공기 중 농도 (㎍/㎥)'] = np.array(conc_df.loc['o5':'o8']).flatten()
table_s['예측된 먼지 농도 (㎍/g)'] = np.array(conc_df.loc['o5':'o8']).flatten()
table_4=pn.Column(table_s,width=1000,height=200)

# 사무실
table_o = pd.DataFrame(columns=['시나리오', '예측된 공기 중 농도 (㎍/㎥)', '예측된 먼지 농도 (㎍/g)'])
table_o['시나리오'] = ['사무용책상 10개, 직물의자 10개, 컴퓨터 10개,벽페인트, 바닥(PVC 타일)', '사무용책상 10개, 가죽의자 10개, 컴퓨터 10개, 벽페인트, 바닥(PVC 타일)']
table_o['예측된 공기 중 농도 (㎍/㎥)'] = np.array(conc_df.loc['o9':'o10']).flatten()
table_o['예측된 먼지 농도 (㎍/g)'] = np.array(conc_df.loc['o9':'o10']).flatten()
table_5=pn.Column(table_o,width=1000,height=200)



In [28]:
def plot_his(x):
    result_df=x
    # fig = px.histogram(df, x=df['exposure'], title='Life expectancy in Canada')
    fig = px.histogram(result_df, x=result_df[result_df['age'] == 1]['exposure'], title='영아 노출량',nbins=100)
    fig.update_layout(width=800,height=400)
    fig.update_xaxes(title="노출량")
    fig.update_yaxes(title="빈도")

    fig2 = px.histogram(result_df, x=result_df[result_df['age'] == 2]['exposure'], title='유아 노출량',nbins=100)
    fig2.update_layout(width=800,height=400)
    fig2.update_xaxes(title="노출량")
    fig2.update_yaxes(title="빈도")

    fig3 = px.histogram(result_df, x=result_df[result_df['age'] == 3]['exposure'], title='미취학아동 노출량',nbins=100)
    fig3.update_layout(width=800,height=400)
    fig3.update_xaxes(title="노출량")
    fig3.update_yaxes(title="빈도")

    fig4 = px.histogram(result_df, x=result_df[result_df['age'] == 4]['exposure'], title='취학아동 노출량',nbins=100)
    fig4.update_layout(width=800,height=400)
    fig4.update_xaxes(title="노출량")
    fig4.update_yaxes(title="빈도")

    fig5 = px.histogram(result_df, x=result_df[result_df['age'] == 5]['exposure'], title='성인 노출량',nbins=100)
    fig5.update_layout(width=800,height=400)
    fig5.update_xaxes(title="노출량")
    fig5.update_yaxes(title="빈도")

    fig6 = px.histogram(result_df, x=result_df['exposure'], title='전체 노출량',nbins=100)
    fig6.update_layout(width=800,height=400)
    fig6.update_xaxes(title="노출량")
    fig6.update_yaxes(title="빈도")
    return pn.Column(fig,fig2,fig3,fig4,fig5,fig6)

# Bar 그래프를 위해서 데이터 프레임 조정
weight_h=weight.loc[['h1','h2','h3','h4']]
weight_h.index=['시나리오1','시나리오2','시나리오3','시나리오4']
weight_n=weight.loc[['o1','o2','o3','o4']]
weight_n.index=['시나리오1','시나리오2','시나리오3','시나리오4']
weight_s=weight.loc[['o5','o6','o7','o8']]
weight_s.index=['시나리오1','시나리오2','시나리오3','시나리오4']
weight_o=weight.loc[['o9','o10']]
weight_o.index=['시나리오1','시나리오2']



def plot_bar(x):
    df=x
    fig = px.bar(df, x=df.index, y=list(df), title="실내공기 오염별 방출량 기여도")
    fig.update_traces(width=0.2)
    fig.update_yaxes(range=[0, 1])
    fig.update_layout(width=550,height=400)
    fig.layout.xaxis.title="시나리오 종류"
    fig.layout.yaxis.title="비율"
    # df=x.copy()
    # count=np.arange(1,df['시나리오'].count()+1)
    # scenario_list = []
    # for i in count:
    #     name = '시나리오'+str(i)
    #     scenario_list.append(name)
    # df['시나리오']=scenario_list
    # fig = px.bar(df, x=df['시나리오'], y=df['예측된 공기 중 농도 (㎍/㎥)'])
    # fig2 = px.bar(df, x=df['시나리오'], y=df['예측된 공기 중 농도 (㎍/㎥)'])
    # fig.update_traces(marker_color='#636EFA',name='공기',showlegend=True,)
    # fig2.update_traces(marker_color='#EF553B',name='미세먼지',showlegend=True,)

    # fig3= make_subplots(specs=[[{"secondary_y": True}]])
    # fig3.add_traces(fig.data+fig2.data)
    # fig3.layout.xaxis.title="시나리오 종류"
    # fig3.layout.yaxis.title=""
    # fig3.layout.title="예측농도"
    # fig3.update_layout(width=800,height=400)
    return fig

def plot_bar2(x):
    df=x
    fig = px.bar(df, x=df.index, y=list(df), title="실내먼지 오염별 방출량 기여도(미완성)")
    fig.update_traces(width=0.2)
    fig.update_yaxes(range=[0, 1])
    fig.update_layout(width=550,height=400)
    fig.layout.xaxis.title="시나리오 종류"
    fig.layout.yaxis.title="비율"

    return fig

def pie_chart(x):
    df=x.copy()
    fig = px.pie(df, values='exposure', names='age', title='Population of European continent')
    fig.update_layout(width=800,height=400)
    return fig

# mark2=pn.pane.Markdown("<br>")
# tabs=pn.Tabs(
#     # ('시나리오',pn.Column(pn.Row(plot_A(result1),mark,table_A(result1)),pie_chart(result1))),
#     # ('어린이집',pn.Column(pn.Row(table_2,plot_bar(table_n)),pie_chart(result2))),
#     ('어린이집',pn.Column(table_2,mark2,pn.Row(plot_bar(weight_n),plot_bar2(weight_n)))),
#     #개인단위 테이블 변경예정
#     ('가정',pn.Column(table_3,mark2,mark2,pn.Row(plot_bar(weight_h),plot_bar2(weight_h)))),
#     ('학교교실',pn.Column(table_4,mark2,pn.Row(plot_bar(weight_s),plot_bar2(weight_s)))),
#     ('사무실',pn.Column(table_5,mark2,pn.Row(plot_bar(weight_o),plot_bar2(weight_o)))),

#     ('개인단위노출량',pn.Column(table_1,mark2,pn.Row(plot_his(result_df)))),#table_1

#     dynamic=True
# )

## SERVE

In [29]:
# template = pn.template.FastListTemplate(
#     site="EHR&C", title="활동공간 과거버전" ,
#     # sidebar=[widget_box],
#     main=[tabs]
# ).servable()

In [1]:
weight_h

NameError: name 'weight_h' is not defined